In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from simulator import Simulator

position = np.array([-0.02693469, 0.01129087, 0.0300001])
force = np.array([-0.5080718, -0.7519938, -0.41996235])
loads = [(position, force)]

radius = 0.005
msh = "meshes/primitives/msh/HexagonBore1_cg2.msh"
simulator = Simulator(msh, contact_radius=radius)
uh = simulator.run(loads)
vm = simulator.compute_vm1(uh)
simulator.plot_vm(vm)

Info    : Reading 'meshes/primitives/msh/HexagonBore1_cg2.msh'...
Info    : 51 entities
Info    : 48878 nodes
Info    : 38492 elements
Info    : Done reading 'meshes/primitives/msh/HexagonBore1_cg2.msh'
Using Lagrange elements of order 2 for simulation.



Widget(value='<iframe src="http://localhost:36639/index.html?ui=P_0x7db6eba8a870_0&reconnect=auto" class="pyvi…

In [2]:
simulator.plot_displacement(uh)

Widget(value='<iframe src="http://localhost:36639/index.html?ui=P_0x7db90218a7b0_1&reconnect=auto" class="pyvi…

In [29]:
simulator.plot_vm_bottom(vm)

Widget(value='<iframe src="http://localhost:36639/index.html?ui=P_0x7db60053bb00_10&reconnect=auto" class="pyv…

In [ ]:
import meshio
import pyvista
from dolfinx import fem, plot


def probe(func: fem.Function, points: np.ndarray, epsilon: float = 1e-6) -> np.ndarray:
    """
    Probe the function `func` at the given `points`.
    Clamps points to be slightly inside the mesh boundaries.
    """
    # Get mesh bounding box
    topology, cell_types, geometry = plot.vtk_mesh(func.function_space)

    # Clamp points to mesh bounds with small epsilon
    mesh_bounds = np.array([geometry.min(axis=0), geometry.max(axis=0)])
    points_clamped = np.clip(
        points,
        mesh_bounds[0] + epsilon,  # min bounds + epsilon
        mesh_bounds[1] - epsilon,  # max bounds - epsilon
    )

    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    bs = func.function_space.dofmap.index_map_bs
    grid.point_data["values"] = func.x.array.real.reshape(-1, bs)

    cloud = pyvista.PolyData(points_clamped)
    samples = cloud.sample(grid)

    return samples.point_data["values"].reshape(-1, bs)


msh = "meshes/primitives/msh/HexagonBore1_cg1.msh"
mesh = meshio.read(msh)
uh_sampled = probe(uh, mesh.points)
vm_sampled = probe(vm, mesh.points)

In [39]:
from graph_builder import GraphBuilder, GraphVisualizer
from utils import msh_to_trimesh

builder = GraphBuilder()
visualizer = GraphVisualizer(msh_to_trimesh(mesh))
g = builder.build(mesh, np.hstack([uh_sampled, vm_sampled]), contacts=loads)
visualizer.displacement(g, debug=True)

Widget(value='<iframe src="http://localhost:36639/index.html?ui=P_0x7db628148890_17&reconnect=auto" class="pyv…

In [36]:
visualizer.stress(g, debug=True)

Contact point: [-0.02693469  0.01129087  0.0300001 ], Force: [-0.5080718  -0.7519938  -0.41996235]


Widget(value='<iframe src="http://localhost:36639/index.html?ui=P_0x7db600539370_15&reconnect=auto" class="pyv…

In [37]:
visualizer.bottom(g)

Widget(value='<iframe src="http://localhost:36639/index.html?ui=P_0x7db62814a2d0_16&reconnect=auto" class="pyv…